In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import itertools
import re
from itertools import product
import operator
from functools import reduce
import polars as pl
#from pandas import option_context
#%pip install openpyxl
%config InlineBackend.figure_format = 'retina'
plt.style.use('seaborn-pastel')
plt.style.use('seaborn-dark')

/var/folders/t5/84slq0fn2gx51rghrcgxpjt80000gn/T/ipykernel_73845/2875002457.py:15: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-pastel')
/var/folders/t5/84slq0fn2gx51rghrcgxpjt80000gn/T/ipykernel_73845/2875002457.py:16: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-dark')


In [1]:
import matplotlib
from matplotlib.colors import ListedColormap
from matplotlib.colors import LinearSegmentedColormap
matplotlib.style.use('dark_background')
color_list = ['white','skyblue','pink', 'tomato', 'slateblue', 'wheat', 'lightgrey', 'limegreen', 'royalblue', 'yellowgreen', 'lightseagreen', 'orchid', 'orange']
cmap = LinearSegmentedColormap.from_list("",color_list)

In [2]:
# Import all the google connexion packages
import os
import google.auth
from google.cloud import bigquery
from google.auth import impersonated_credentials

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/Users/a20015173/.config/gcloud/legacy_credentials/ekaterina.rezanovich@adeo.com/adc.json"
client_1 = bigquery.Client(project = 'ccdp-transverse-data-dev')

NPS DATA

In [3]:
# questions data
filepath_1 = "./datasets/questions.parquet"
if not os.path.isfile(filepath_1):

    query = """
    select t1.survey_id, t1.name , t2.question_id, t2.name, t2.content
    from `dfdp-customer-exp-mt-prod.customer_experience_measure_bu001.client_experience_survey` as t1
    left join `dfdp-customer-exp-mt-prod.customer_experience_measure_bu001.client_experience_question` as t2 on t2.survey_id = t1.survey_id
    where t2.question_id in ('SV_81jKyfOtepO6Weq__QID1_NPS_GROUP', 'SV_4OUB8z9FwEgBiU6__QID1_NPS_GROUP', 'SV_5nFeSHRTsx1yxWC__QID1_NPS_GROUP', 'SV_77FLHcnyIyMM07Q__QID1_NPS_GROUP', 'SV_bj9J6UgbM2yIATQ__QID13_NPS_GROUP',
    'SV_2fA21joHLvPEZoi__QID1_NPS_GROUP', 'SV_5nVqXD824PZXj14__QID1_NPS_GROUP')"""

    dl_data = pd.read_gbq(query=query, project_id="ccdp-transverse-data-dev")
    dl_data.to_parquet(filepath_1)

df_q = pd.read_parquet(filepath_1)

In [4]:
# answers data
filepath_2 = "./datasets/answers.parquet"
if not os.path.isfile(filepath_2):

    query = """
select t3.answer_id, t3.feedback_id, t3.question_id, t3.score, t3.score_category, t3.comment,
t4.touchpoint_id, t4.store_code, t4.user_id, t4.user_id_type, cast(t4.date as date) as answer_date
from `dfdp-customer-exp-mt-prod.customer_experience_measure_bu001.client_experience_answer` as t3
left join `dfdp-customer-exp-mt-prod.customer_experience_measure_bu001.client_experience_feedback` as t4 on t4.feedback_id = t3.feedback_id
where t3.question_id in ('SV_81jKyfOtepO6Weq__QID1_NPS_GROUP', 'SV_4OUB8z9FwEgBiU6__QID1_NPS_GROUP', 'SV_5nFeSHRTsx1yxWC__QID1_NPS_GROUP', 'SV_77FLHcnyIyMM07Q__QID1_NPS_GROUP', 'SV_bj9J6UgbM2yIATQ__QID13_NPS_GROUP',
'SV_2fA21joHLvPEZoi__QID1_NPS_GROUP', 'SV_5nVqXD824PZXj14__QID1_NPS_GROUP')
    """

    dl_data = pd.read_gbq(query=query, project_id="ccdp-transverse-data-dev")
    dl_data.to_parquet(filepath_2)

df_a = pd.read_parquet(filepath_2)

In [5]:
df_nps_all = df_a.merge(df_q, left_on = 'question_id', right_on = 'question_id', how='left')

In [6]:
del df_nps_all['name_1'], df_nps_all['user_id_type'],  df_nps_all['content']

In [7]:
# filter only nps with user_id
df_nps = df_nps_all[df_nps_all['user_id'].notna()]

In [8]:
# first and last TP of the customer
df_nps_dates = df_nps.groupby('user_id')['answer_date'].agg(['min', 'max']).reset_index()
df_nps_dates.columns = ['uid', 'first_nps_date', 'last_nps_date']

In [9]:
df_nps_1 = df_nps.merge(df_nps_dates, left_on='user_id', right_on='uid', how='left')
del df_nps_1['uid']

In [10]:
# nb FB of the customer
df_nps_nb_fb = df_nps.groupby('user_id')['feedback_id'].nunique().reset_index()
df_nps_nb_fb.columns = ['uid', 'nb_fb']

In [11]:
df_nps_2 = df_nps_1.merge(df_nps_nb_fb, left_on='user_id', right_on='uid', how='left')
del df_nps_2['uid']

In [12]:
# 0 for detractor, 1 for passive, 2 for promotor
def fb_type(df):
    if df['score'] <= 6 :
        return 0
    elif df['score'] >= 9:
        return 2
    else:
        return 1

In [13]:
df_nps_2['score'].unique()

<IntegerArray>
[8, 10, 9, 4, 0, 3, 6, 5, 1, 7, 2, <NA>]
Length: 12, dtype: Int64

In [14]:
df_nps_3 = df_nps_2[df_nps_2['score'].notna()]

In [15]:
df_nps_3['fb_type'] = df_nps_3.apply(fb_type, axis=1)

/var/folders/t5/84slq0fn2gx51rghrcgxpjt80000gn/T/ipykernel_73845/761178500.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nps_3['fb_type'] = df_nps_3.apply(fb_type, axis=1)


In [16]:
# types of the customer fb
df_nps_fb_typ = df_nps_3.pivot_table(index='user_id', columns='fb_type', values='feedback_id', aggfunc='nunique').reset_index()
df_nps_fb_typ.columns = ['uid', '0', '1', '2']
df_nps_fb_typ = df_nps_fb_typ.fillna(0)
df_nps_fb_typ.head()

,uid,0,1,2
0,1000000,0.0,0.0,2.0
1,10000024,1.0,0.0,0.0
2,1000004,0.0,0.0,1.0
3,1000011,2.0,0.0,0.0
4,10000188,0.0,0.0,1.0


In [17]:
def fb_type_res(df):
    if df['0']==0 and df['1']==0:
        return "only positive"
    elif df['0']==0 and df['2']==0:
        return "only passive"
    elif df['1']==0 and df['2']==0:
        return "only negative"
    elif df['0']==0:
        return "passive and positive"
    elif df['1']==0:
        return "negative and positive"  
    elif df['2']==0:
        return "negative and passive"  
    else:
        return "negative, passive, positive"

In [18]:
df_nps_fb_typ['fb_type'] = df_nps_fb_typ.apply(fb_type_res, axis=1)

In [19]:
def fb_type_res_nb(df):
    if (df['0']+df['1']+df['2']) > 1:
        return "mutli"
    else:
        return "mono"

In [20]:
df_nps_fb_typ['fb_nb_type'] = df_nps_fb_typ.apply(fb_type_res_nb, axis=1)

In [21]:
df_nps_fb_typ.columns = ['uid', 'nb_fb_neg', 'nb_fb_pas', 'nb_fb_pos', 'fb_group', 'fb_nb_type']

In [22]:
df_nps_4 = df_nps_3.merge(df_nps_fb_typ, left_on='user_id', right_on='uid', how='left')
del df_nps_4['uid']

In [23]:
df_nps_4.head(2)

,answer_id,feedback_id,question_id,score,score_category,comment,touchpoint_id,store_code,user_id,answer_date,...,name,first_nps_date,last_nps_date,nb_fb,fb_type,nb_fb_neg,nb_fb_pas,nb_fb_pos,fb_group,fb_nb_type
0,R_111w3BVzUYwknmY__QID1_NPS_GROUP,R_111w3BVzUYwknmY,SV_81jKyfOtepO6Weq__QID1_NPS_GROUP,8,Passif,"Bonjour, \nJe suis entièrement satisfait de ...",SV_81jKyfOtepO6Weq,<NA>,10217876,2023-08-20,...,LMFR - Client SAV - PROD,2022-04-09,2023-08-20,14,1,0.0,1.0,13.0,passive and positive,mutli
1,R_3htf0IgZ0ecDb8u__QID1_NPS_GROUP,R_3htf0IgZ0ecDb8u,SV_81jKyfOtepO6Weq__QID1_NPS_GROUP,10,Promoteur,None,SV_81jKyfOtepO6Weq,<NA>,54974624,2023-08-16,...,LMFR - Client SAV - PROD,2023-08-16,2023-08-16,1,2,0.0,0.0,1.0,only positive,mono


In [24]:
df_nps_summary_type = df_nps_4.groupby('fb_nb_type')['user_id'].nunique().reset_index()
df_nps_summary_type['share_%'] = (df_nps_summary_type['user_id'] / df_nps_summary_type['user_id'].sum())*100
df_nps_summary_type

,fb_nb_type,user_id,share_%
0,mono,889117,72.614886
1,mutli,335311,27.385114


In [25]:
df_nps_summary_group = df_nps_4.groupby('fb_group')['user_id'].nunique().reset_index()
df_nps_summary_group['share_%'] = (df_nps_summary_group['user_id'] / df_nps_summary_group['user_id'].sum())*100
df_nps_summary_group = df_nps_summary_group.sort_values(by='share_%', ascending=False)
df_nps_summary_group

,fb_group,user_id,share_%
5,only positive,680303,55.560882
4,only passive,277548,22.667564
3,only negative,129318,10.561503
6,passive and positive,77828,6.356274
1,negative and positive,26855,2.193269
0,negative and passive,21942,1.792020
2,"negative, passive, positive",10634,0.868487


In [26]:
df_nps_summary = df_nps_4.groupby(['fb_nb_type', 'fb_group'])['user_id'].nunique().reset_index()
df_nps_summary['share_%'] = (df_nps_summary['user_id'] / df_nps_summary['user_id'].sum())*100
df_nps_summary = df_nps_summary.sort_values(by=['fb_nb_type','share_%'], ascending=[True, False])
df_nps_summary

,fb_nb_type,fb_group,user_id,share_%
2,mono,only positive,524428,42.830448
1,mono,only passive,246769,20.153819
0,mono,only negative,117920,9.630619
8,mutli,only positive,155875,12.730434
9,mutli,passive and positive,77828,6.356274
7,mutli,only passive,30779,2.513745
4,mutli,negative and positive,26855,2.193269
3,mutli,negative and passive,21942,1.792020
6,mutli,only negative,11398,0.930884
5,mutli,"negative, passive, positive",10634,0.868487


SALES DATA

In [56]:
df_s0 = pd.read_parquet('./datasets/sales_gr/df_s_3p.parquet')
df_s1 = pd.read_parquet('./datasets/sales_gr/df_s_22_1_1p.parquet')

In [28]:
df_s2 = pd.read_parquet('./datasets/sales_gr/df_s_22_2_1p.parquet')

In [29]:
df_s3 = pd.read_parquet('./datasets/sales_gr/df_s_22_3_1p.parquet')

In [30]:
df_s4 = pd.read_parquet('./datasets/sales_gr/df_s_22_4_1p.parquet')

In [31]:
df_s5 = pd.read_parquet('./datasets/sales_gr/df_s_23_1_1p.parquet')

In [32]:
df_s6 = pd.read_parquet('./datasets/sales_gr/df_s_23_2_1p.parquet')

In [33]:
df_s7 = pd.read_parquet('./datasets/sales_gr/df_s_23_3_1p.parquet')

In [57]:
df_s_1p = df_s0.append([df_s1,df_s2,df_s3])

/var/folders/t5/84slq0fn2gx51rghrcgxpjt80000gn/T/ipykernel_73845/2518559154.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_s_1p = df_s0.append([df_s1,df_s2,df_s3])


In [35]:
df_s_2p = df_s4.append([df_s5,df_s6,df_s7])

/var/folders/t5/84slq0fn2gx51rghrcgxpjt80000gn/T/ipykernel_73845/3628812971.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_s_2p = df_s4.append([df_s5,df_s6,df_s7])


In [58]:
df_s = df_s_1p.append(df_s_2p)

/var/folders/t5/84slq0fn2gx51rghrcgxpjt80000gn/T/ipykernel_73845/4204960968.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_s = df_s_1p.append(df_s_2p)


In [59]:
df_s.head()

,store_id,cart_date,cart_id,user_id,top_l_c,sales_channel,delivery_channel,amount_to,qty
0,380,2022-01-01,001-20350L606-A,56210348,0,Website,Delivery,23.40,1
1,380,2022-01-01,001-21023L763-A,77795876,0,Website,Delivery,42.90,1
2,380,2022-01-01,001-21080L164-A,67739822,0,Website,Delivery,104.99,1
3,380,2022-01-01,001-21089L571-A,70027761,0,Website,Delivery,25.45,1
4,380,2022-01-01,001-21209L741-A,64195109,0,Website,Delivery,27.99,1


In [38]:
df_s['cart_date'].min()

datetime.date(2022, 1, 1)

In [51]:
df_s['cart_date'].max()

datetime.date(2023, 10, 19)

In [40]:
df_s['store_id'].unique()

<IntegerArray>
[  3,   4,   5,   6,  10,  11,  12,  13,  14,  15,
 ...
 291, 292, 293, 294, 295, 296, 297, 298, 380, 230]
Length: 146, dtype: Int64

In [41]:
df_s['sales_channel'].unique()

array(['Store', 'Website'], dtype=object)

In [42]:
df_s['delivery_channel'].unique()

array(['Self-service', 'Pickup', 'Self-service - Sales Bulletin',
       'Delivery'], dtype=object)

In [60]:
df_s['user_id'].nunique()

9567696

In [62]:
# filter nps data for clear user_id
no_user_id = ['undefined', 'a', 'ra']
df_nps = df_nps.query('user_id not in @no_user_id')
df_nps['user_id'] = df_nps['user_id'].astype('int')

In [63]:
df_nps_users = df_nps.groupby('user_id')['feedback_id'].nunique().reset_index()
nps_users = df_nps_users['user_id'].to_list()

In [64]:
# filter users with nps reviews
df_s_nps = df_s.query('user_id in @nps_users')

In [65]:
print("Nb customers with purchase", df_s['user_id'].nunique())
print("Nb customers with purchase and nps", df_s_nps['user_id'].nunique())

Nb customers with purchase 9567696
Nb customers with purchase and nps 1216062


In [67]:
1216062 / 9567696

0.12710081925679914

In [68]:
# first,last dates of purchases by channel
df_s_nps_ch_dates = df_s_nps.pivot_table(index='user_id', columns = 'sales_channel',values='cart_date',\
                                          aggfunc=['min','max']).reset_index()
df_s_nps_ch_dates.columns = ['uid', 'min_date_store', 'min_date_web', 'max_date_store', 'max_date_web']
df_s_nps_ch_dates.head(2)

,uid,min_date_store,min_date_web,max_date_store,max_date_web
0,14,2022-02-12,2022-03-08,2023-09-05,2023-07-15
1,15,2022-02-09,2022-01-03,2023-10-07,2023-10-10


In [72]:
# first,last dates of purchases in general
df_s_nps_dates = df_s_nps.groupby('user_id').agg({"cart_date": ['min', 'max']}).reset_index()
df_s_nps_dates.columns = ['uid', 'min_date', 'max_date']
df_s_nps_dates.head(2)

,uid,min_date,max_date
0,14,2022-02-12,2023-09-05
1,15,2022-01-03,2023-10-10


In [73]:
#merge this info to sales table
df_s_2 = df_s_nps.merge(df_s_nps_ch_dates, left_on='user_id', right_on='uid', how='left')
del df_s_2['uid']
df_s_2.head(2)

,store_id,cart_date,cart_id,user_id,top_l_c,sales_channel,delivery_channel,amount_to,qty,min_date_store,min_date_web,max_date_store,max_date_web
0,380,2022-01-01,001-21209L741-A,64195109,0,Website,Delivery,27.99,1,2023-08-11,2022-01-01,2023-08-11,2023-10-08
1,380,2022-01-01,001-21222L1732-A,55072654,0,Website,Delivery,249.00,1,2022-01-05,2022-01-01,2022-05-13,2023-07-20


In [74]:
df_s_3 = df_s_2.merge(df_s_nps_dates, left_on='user_id', right_on='uid', how='left')
del df_s_3['uid']
df_s_3.head(2)

,store_id,cart_date,cart_id,user_id,top_l_c,sales_channel,delivery_channel,amount_to,qty,min_date_store,min_date_web,max_date_store,max_date_web,min_date,max_date
0,380,2022-01-01,001-21209L741-A,64195109,0,Website,Delivery,27.99,1,2023-08-11,2022-01-01,2023-08-11,2023-10-08,2022-01-01,2023-10-08
1,380,2022-01-01,001-21222L1732-A,55072654,0,Website,Delivery,249.00,1,2022-01-05,2022-01-01,2022-05-13,2023-07-20,2022-01-01,2023-07-20


In [75]:
df_nps_5 = df_nps_4.query('user_id not in @no_user_id')
df_nps_5['uid'] = df_nps_5['user_id'].astype('int')

/var/folders/t5/84slq0fn2gx51rghrcgxpjt80000gn/T/ipykernel_73845/692595386.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nps_5['uid'] = df_nps_5['user_id'].astype('int')


In [76]:
df_nps_6 = df_nps_5.groupby(['uid', 'first_nps_date', 'last_nps_date', 'nb_fb_neg', 'nb_fb_pas','nb_fb_pos','fb_group',\
                                 'fb_nb_type'])['nb_fb'].mean().reset_index()

In [77]:
df_nps_6.head()

,uid,first_nps_date,last_nps_date,nb_fb_neg,nb_fb_pas,nb_fb_pos,fb_group,fb_nb_type,nb_fb
0,14,2022-05-04,2022-05-04,0.0,1.0,0.0,only passive,mono,1.0
1,15,2022-11-13,2023-01-22,0.0,0.0,2.0,only positive,mutli,2.0
2,16,2023-02-07,2023-02-07,0.0,0.0,1.0,only positive,mono,1.0
3,19,2022-06-29,2022-11-15,0.0,1.0,1.0,passive and positive,mutli,2.0
4,22,2022-07-05,2023-07-19,0.0,2.0,1.0,passive and positive,mutli,3.0


In [79]:
df_s_3['cart_id'].nunique()

25961354

In [85]:
# users' purchases characterisrics
df_s_ch = df_s_3.pivot_table(index='user_id', columns='sales_channel', 
                             aggfunc={'amount_to' : 'sum', 'qty' : 'sum', 'cart_id':'nunique'}).reset_index()

In [86]:
df_s_ch.columns = ['user_id','amount_to_store','amount_to_web','nb_carts_store', 'nb_carts_web', 'qty_store','qty_web']
df_s_ch.head(2)

,user_id,amount_to_store,amount_to_web,nb_carts_store,nb_carts_web,qty_store,qty_web
0,14,315.51,341.01,20.0,4.0,34.0,9.0
1,15,5968.82,1072.00,67.0,14.0,273.0,82.0


In [87]:
df_s_nps_ch_dates.head(2)

,uid,min_date_store,min_date_web,max_date_store,max_date_web
0,14,2022-02-12,2022-03-08,2023-09-05,2023-07-15
1,15,2022-02-09,2022-01-03,2023-10-07,2023-10-10


In [88]:
df_s_nps_dates.head(2)

,uid,min_date,max_date
0,14,2022-02-12,2023-09-05
1,15,2022-01-03,2023-10-10


In [89]:
df_nps_6.head(2)

,uid,first_nps_date,last_nps_date,nb_fb_neg,nb_fb_pas,nb_fb_pos,fb_group,fb_nb_type,nb_fb
0,14,2022-05-04,2022-05-04,0.0,1.0,0.0,only passive,mono,1.0
1,15,2022-11-13,2023-01-22,0.0,0.0,2.0,only positive,mutli,2.0


In [90]:
df_s_4 = df_s_ch.merge(df_s_nps_ch_dates, left_on='user_id', right_on='uid', how='left')

In [91]:
df_s_5 = df_s_4.merge(df_s_nps_dates, left_on='user_id', right_on='uid', how='left')

In [156]:
df_s_6 = df_s_5.merge(df_nps_6, left_on='user_id', right_on='uid', how='left')

In [150]:
del df_s_6['uid_x'], df_s_6['uid_y'], df_s_6['uid']

In [152]:
df_s_6.to_parquet('./datasets/sales_summary_nps.parquet')

In [27]:
df_s_6 = pd.read_parquet('./datasets/sales_summary_nps.parquet')

In [97]:
print("Nb customers with purchase", df_s['user_id'].nunique())
print("Nb customers with purchase and nps", df_s_6['user_id'].nunique())

Nb customers with purchase 9567696
Nb customers with purchase and nps 1216062


In [98]:
print('% of customers with purchases and NPS: ', (df_s_6['user_id'].nunique()/df_s['user_id'].nunique())*100)

% of customers with purchases and NPS:  12.710081925679914


### 13 % of all customers that did purchases (web, store, both) gave NPS

### Check why 1->0 !!!!

In [163]:
df_s_6[df_s_6['user_id']==96972383]

,user_id,amount_to_store,amount_to_web,nb_carts_store,nb_carts_web,qty_store,qty_web,uid_x,min_date_store,min_date_web,...,max_date,uid,first_nps_date,last_nps_date,nb_fb_neg,nb_fb_pas,nb_fb_pos,fb_group,fb_nb_type,nb_fb
1216057,96972383,NaN,61.85,NaN,1.0,NaN,5.0,96972383,NaT,2023-09-27,...,2023-09-27,96972383.0,2023-09-16,2023-09-16,0.0,1.0,0.0,only passive,mono,1.0


In [164]:
#quality check store
def q_check(df):
    #ordinary cases
    if ((~pd.isna(df['nb_carts_store'])) and (~pd.isna(df['amount_to_store'])) and (~pd.isna(df['qty_store']))) or \
        (pd.isna(df['nb_carts_store']) and pd.isna(df['amount_to_store']) and pd.isna(df['qty_store'])) or \
        ((~pd.isna(df['nb_carts_web'])) and (~pd.isna(df['amount_to_web'])) and (~pd.isna(df['qty_web']))) or \
        (pd.isna(df['nb_carts_web']) and pd.isna(df['amount_to_web']) and pd.isna(df['qty_web'])) or \
        (((~pd.isna(df['nb_carts_store'])) and (~pd.isna(df['amount_to_store'])) and (~pd.isna(df['qty_store']))) and \
        ((~pd.isna(df['nb_carts_web'])) and (~pd.isna(df['amount_to_web'])) and (~pd.isna(df['qty_web'])))):
        return 1
    # return the items cases
    elif ((~pd.isna(df['nb_carts_store'])) and pd.isna(df['amount_to_store']) and pd.isna(df['qty_store'])) or\
         ((~pd.isna(df['nb_carts_web'])) and pd.isna(df['amount_to_web']) and pd.isna(df['qty_web'])):
        return 2
    else:
        return 0

Note: carts with both purchases anf return, do not take into account amount and qty for this part of the analysis

In [165]:
df_s_6['q_ch'] = df_s_6.apply(q_check, axis=1)

Continue check from here !


In [108]:
def type_purchase(df):
    #customer did only web purchase
    if pd.isnull(df['min_date_store']) and df['min_date_web'] is not pd.NaT:
        return "only web"
    elif pd.isnull(df['min_date_web']) and df['min_date_store'] is not pd.NaT:
        return "only store"
    elif df['min_date_store'] is not pd.NaT and df['min_date_web'] is not pd.NaT:
        return "store, web"
    else:
        return "?"

In [109]:
df_s_6['purchase_type'] = df_s_6.apply(type_purchase, axis=1)

In [138]:
df_s_6[df_s_6['purchase_type']=='only web']

,user_id,amount_to_store,amount_to_web,nb_carts_store,nb_carts_web,qty_store,qty_web,min_date_store,min_date_web,max_date_store,...,nb_fb_pas,nb_fb_pos,fb_group,fb_nb_type,nb_fb,q_ch,purchase_type,last_purchase,return_flag,purchase_nb
53,717,0.0,116.99,0.0,0.0,0.0,2.0,NaT,2022-12-01,NaT,...,1.0,0.0,only passive,mono,1.0,1,only web,only web,return,mono purchase
79,1111,0.0,387.00,0.0,0.0,0.0,33.0,NaT,2023-08-17,NaT,...,1.0,1.0,passive and positive,mutli,2.0,1,only web,only web,return,mono purchase
86,1181,0.0,129.00,0.0,0.0,0.0,2.0,NaT,2022-12-17,NaT,...,0.0,1.0,only positive,mono,1.0,1,only web,only web,return,mono purchase
113,1523,0.0,1165.74,0.0,0.0,0.0,14.0,NaT,2023-04-18,NaT,...,0.0,1.0,only positive,mono,1.0,1,only web,only web,return,mono purchase
120,1691,0.0,55.17,0.0,0.0,0.0,7.0,NaT,2022-04-01,NaT,...,0.0,1.0,only positive,mono,1.0,1,only web,only web,return,mono purchase
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1216057,96972383,0.0,61.85,0.0,0.0,0.0,5.0,NaT,2023-09-27,NaT,...,1.0,0.0,only passive,mono,1.0,1,only web,only web,return,mono purchase
1216058,96972402,0.0,125.95,0.0,0.0,0.0,1.0,NaT,2023-09-16,NaT,...,1.0,0.0,only passive,mono,1.0,1,only web,only web,no return,mono purchase
1216059,96972430,0.0,42.90,0.0,0.0,0.0,2.0,NaT,2023-09-18,NaT,...,0.0,1.0,only positive,mono,1.0,1,only web,only web,return,mono purchase
1216060,96972625,0.0,386.90,0.0,0.0,0.0,3.0,NaT,2023-09-18,NaT,...,0.0,0.0,only negative,mono,1.0,1,only web,only web,return,mono purchase


In [110]:
def last_purchase(df):
    if df['purchase_type'] == "store, web":
        if df['max_date_store'] >= df['max_date_web']:
            return "last store purchase"
        else:
            return "last web purchase" 
    else:
        return df['purchase_type']

In [111]:
df_s_6['last_purchase'] = df_s_6.apply(last_purchase, axis=1)

In [112]:
df_s_6.head(5)

,user_id,amount_to_store,amount_to_web,nb_carts_store,nb_carts_web,qty_store,qty_web,min_date_store,min_date_web,max_date_store,...,last_nps_date,nb_fb_neg,nb_fb_pas,nb_fb_pos,fb_group,fb_nb_type,nb_fb,q_ch,purchase_type,last_purchase
0,14,315.51,341.01,20.0,20.0,34.0,9.0,2022-02-12,2022-03-08,2023-09-05,...,2022-05-04,0.0,1.0,0.0,only passive,mono,1.0,1,"store, web",last store purchase
1,15,5968.82,1072.00,67.0,67.0,273.0,82.0,2022-02-09,2022-01-03,2023-10-07,...,2023-01-22,0.0,0.0,2.0,only positive,mutli,2.0,1,"store, web",last web purchase
2,16,1656.04,35.79,53.0,53.0,118.0,2.0,2022-02-14,2023-02-07,2023-10-11,...,2023-02-07,0.0,0.0,1.0,only positive,mono,1.0,1,"store, web",last store purchase
3,19,5674.13,0.00,34.0,34.0,244.0,0.0,2022-05-05,NaT,2023-10-19,...,2022-11-15,0.0,1.0,1.0,passive and positive,mutli,2.0,1,only store,only store
4,22,3663.47,0.00,32.0,32.0,36.0,0.0,2022-01-05,NaT,2023-07-20,...,2023-07-19,0.0,2.0,1.0,passive and positive,mutli,3.0,1,only store,only store


In [113]:
def return_flag(df):
    # only web customer
    if df['purchase_type'] == 'only web':
        if df['last_nps_date'] >= df['max_date_web']:
            return "no return"
        else:
            return "return"
    # only store customer
    elif df['purchase_type'] == 'only store':
        if df['last_nps_date'] >= df['max_date_store']:
            return "no return"
        else:
            return "return"
    # store, web customer
    elif df['purchase_type'] == 'store, web':
        # last purchase web
        if df['last_purchase'] == 'last web purchase':
            if df['last_nps_date'] >= df['max_date_web']:
                return "no return"
            else:
                if df['last_nps_date'] >= df['max_date_store']:
                    return "return/ store->web"
                else:
                    return "return/ ->store,web"
        #lasr purchase store
        if df['last_purchase'] == 'last store purchase':
            if df['last_nps_date'] >= df['max_date_store']:
                return "no return"
            else:
                if df['last_nps_date'] >= df['max_date_web']:
                    return "return/ web->store"
                else:
                    return "return/ ->web,store"

In [114]:
df_s_6['return_flag'] = df_s_6.apply(return_flag, axis=1)

/var/folders/t5/84slq0fn2gx51rghrcgxpjt80000gn/T/ipykernel_73845/4113624140.py:27: FutureWarning: Comparison of NaT with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable.
  if df['last_nps_date'] >= df['max_date_store']:
/var/folders/t5/84slq0fn2gx51rghrcgxpjt80000gn/T/ipykernel_73845/4113624140.py:30: FutureWarning: Comparison of NaT with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable.
  if df['last_nps_date'] >= df['max_date_web']:
/var/folders/t5/84slq0fn2gx51rghrcgxpjt80000gn/T/ipykernel_73845/4113624140.py:10: FutureWarning: Comparison of NaT with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable.
  if df['last_nps_date'] >= df['max_date_store']:
/var/folders/t5/84slq0fn2gx51rghrcgxpjt80000gn/T/ipykernel_73845/4113624140

In [117]:
df_s_6.to_parquet('./datasets/first_results.parquet')

In [134]:
def nb_purchase(df):
    if (df['nb_carts_store']+df['nb_carts_web'])>1:
        return "multi purchase"
    else:
        return "mono purchase"

In [135]:
df_s_6['purchase_nb'] = df_s_6.apply(nb_purchase, axis=1)

In [136]:
df_s_6[df_s_6['purchase_nb']=='multi purchase']

,user_id,amount_to_store,amount_to_web,nb_carts_store,nb_carts_web,qty_store,qty_web,min_date_store,min_date_web,max_date_store,...,nb_fb_pas,nb_fb_pos,fb_group,fb_nb_type,nb_fb,q_ch,purchase_type,last_purchase,return_flag,purchase_nb
0,14,315.51,341.01,20.0,20.0,34.0,9.0,2022-02-12,2022-03-08,2023-09-05,...,1.0,0.0,only passive,mono,1.0,1,"store, web",last store purchase,"return/ ->web,store",multi purchase
1,15,5968.82,1072.00,67.0,67.0,273.0,82.0,2022-02-09,2022-01-03,2023-10-07,...,0.0,2.0,only positive,mutli,2.0,1,"store, web",last web purchase,"return/ ->store,web",multi purchase
2,16,1656.04,35.79,53.0,53.0,118.0,2.0,2022-02-14,2023-02-07,2023-10-11,...,0.0,1.0,only positive,mono,1.0,1,"store, web",last store purchase,return/ web->store,multi purchase
3,19,5674.13,0.00,34.0,34.0,244.0,0.0,2022-05-05,NaT,2023-10-19,...,1.0,1.0,passive and positive,mutli,2.0,1,only store,only store,return,multi purchase
4,22,3663.47,0.00,32.0,32.0,36.0,0.0,2022-01-05,NaT,2023-07-20,...,2.0,1.0,passive and positive,mutli,3.0,1,only store,only store,return,multi purchase
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1215954,96967036,206.90,99.00,4.0,4.0,11.0,1.0,2023-09-16,2023-09-16,2023-10-12,...,1.0,0.0,only passive,mono,1.0,1,"store, web",last store purchase,return/ web->store,multi purchase
1216011,96970451,-14.90,55.80,1.0,1.0,-1.0,3.0,2023-09-22,2023-09-17,2023-09-22,...,0.0,1.0,only positive,mono,1.0,1,"store, web",last store purchase,"return/ ->web,store",multi purchase
1216013,96970479,-79.80,79.80,1.0,1.0,-2.0,2.0,2023-09-17,2023-09-17,2023-09-17,...,0.0,1.0,only positive,mono,1.0,1,"store, web",last store purchase,"return/ ->web,store",multi purchase
1216028,96971283,77.00,198.78,2.0,2.0,5.0,4.0,2023-09-23,2023-09-16,2023-10-14,...,1.0,0.0,only passive,mono,1.0,1,"store, web",last store purchase,"return/ ->web,store",multi purchase


In [121]:
df_s_6['user_id'].nunique()

1216062

In [122]:
df_s_6.shape[0]

1216062

### Summary by NPS

In [123]:
df_nps_summary_type = df_s_6.groupby('fb_nb_type')['user_id'].nunique().reset_index()
df_nps_summary_type['share_%'] = (df_nps_summary_type['user_id'] / df_nps_summary_type['user_id'].sum())*100
df_nps_summary_type

,fb_nb_type,user_id,share_%
0,mono,880891,72.439132
1,mutli,335152,27.560868


In [124]:
df_nps_summary_group = df_s_6.groupby('fb_group')['user_id'].nunique().reset_index()
df_nps_summary_group['share_%'] = (df_nps_summary_group['user_id'] / df_nps_summary_group['user_id'].sum())*100
df_nps_summary_group = df_nps_summary_group.sort_values(by='share_%', ascending=False)
df_nps_summary_group

,fb_group,user_id,share_%
5,only positive,675251,55.528546
4,only passive,275335,22.641880
3,only negative,128240,10.545680
6,passive and positive,77806,6.398293
1,negative and positive,26847,2.207734
0,negative and passive,21930,1.803390
2,"negative, passive, positive",10634,0.874476


In [125]:
df_nps_summary = df_s_6.groupby(['fb_nb_type', 'fb_group'])['user_id'].nunique().reset_index()
df_nps_summary['share_%'] = (df_nps_summary['user_id'] / df_nps_summary['user_id'].sum())*100
df_nps_summary = df_nps_summary.sort_values(by=['fb_nb_type','share_%'], ascending=[True, False])
df_nps_summary

,fb_nb_type,fb_group,user_id,share_%
2,mono,only positive,519465,42.717651
1,mono,only passive,244574,20.112282
0,mono,only negative,116852,9.609200
8,mutli,only positive,155786,12.810896
9,mutli,passive and positive,77806,6.398293
7,mutli,only passive,30761,2.529598
4,mutli,negative and positive,26847,2.207734
3,mutli,negative and passive,21930,1.803390
6,mutli,only negative,11388,0.936480
5,mutli,"negative, passive, positive",10634,0.874476


### Summary by Sales

In [126]:
df_s_nps_mm = df_s_6.groupby('purchase_nb')['user_id'].nunique().reset_index()
df_s_nps_mm.columns = ['purchase_nb', 'nb_users']
df_s_nps_mm['share_%'] = (df_s_nps_mm['nb_users'] / df_s_nps_mm['nb_users'].sum()*100).round(2)
df_s_nps_mm

,purchase_nb,nb_users,share_%
0,mono purchase,338559,27.84
1,multi purchase,877503,72.16


In [129]:
df_s_nps_pt = df_s_6.groupby('purchase_type')['user_id'].nunique().reset_index()
df_s_nps_pt.columns = ['purchase_type', 'nb_users']
df_s_nps_pt['share_%'] = (df_s_nps_pt['nb_users'] / df_s_nps_pt['nb_users'].sum()*100).round(2)
df_s_nps_pt = df_s_nps_pt.sort_values(by='share_%', ascending=False)
df_s_nps_pt

,purchase_type,nb_users,share_%
2,"store, web",511487,42.06
0,only store,366016,30.10
1,only web,338559,27.84


In [131]:
df_s_nps_mm_t = df_s_6.groupby(['purchase_nb', 'purchase_type'])['user_id'].nunique().reset_index()
df_s_nps_mm_t.columns = ['purchase_nb', 'purchase_type', 'nb_users']
df_s_nps_mm_t['share_%'] = (df_s_nps_mm_t['nb_users'] / df_s_nps_mm_t['nb_users'].sum()*100).round(2)
df_s_nps_mm_t = df_s_nps_mm_t.sort_values(by='share_%', ascending=False)
df_s_nps_mm_t

,purchase_nb,purchase_type,nb_users,share_%
2,multi purchase,"store, web",511487,42.06
1,multi purchase,only store,366016,30.10
0,mono purchase,only web,338559,27.84


In [133]:
df_s_6[(df_s_6['purchase_nb']=='multi purchase')&(df_s_6['purchase_type']=='only web')]

,user_id,amount_to_store,amount_to_web,nb_carts_store,nb_carts_web,qty_store,qty_web,min_date_store,min_date_web,max_date_store,...,nb_fb_pas,nb_fb_pos,fb_group,fb_nb_type,nb_fb,q_ch,purchase_type,last_purchase,return_flag,purchase_nb


In [130]:
df_s_nps_ret = df_s_6.groupby('return_flag')['user_id'].nunique().reset_index()
df_s_nps_ret.columns = ['return_flag', 'nb_users']
df_s_nps_ret['share_%'] = (df_s_nps_ret['nb_users'] / df_s_nps_ret['nb_users'].sum()*100).round(2)
df_s_nps_ret = df_s_nps_ret.sort_values(by='share_%', ascending=False)
df_s_nps_ret

,return_flag,nb_users,share_%
1,return,505064,41.53
0,no return,242696,19.96
3,"return/ ->web,store",239121,19.66
5,return/ web->store,131333,10.80
2,"return/ ->store,web",52322,4.30
4,return/ store->web,45526,3.74


In [148]:
df_s_6[(df_s_6['purchase_nb']=='mono purchase')&(df_s_6['purchase_type']=='store, web')]

,user_id,top_l_c,amount_to_store,amount_to_web,nb_carts_store,nb_carts_web,qty_store,qty_web,min_date_store,min_date_web,...,nb_fb,nb_fb_neg,nb_fb_pas,nb_fb_pos,fb_group,fb_nb_type,purchase_type,last_purchase,return_flag,purchase_nb
106,679,0,0.0,53.10,0.0,0.0,0.0,7.0,2023-10-08,2022-06-02,...,2.0,0.0,0.0,2.0,only positive,mutli,"store, web",last store purchase,"return/ ->web,store",mono purchase
107,679,0,0.0,53.10,0.0,0.0,0.0,7.0,2023-10-08,2022-06-02,...,2.0,0.0,0.0,2.0,only positive,mutli,"store, web",last store purchase,"return/ ->web,store",mono purchase
722,5999,0,0.0,490.60,0.0,0.0,0.0,22.0,2022-01-11,2022-03-17,...,1.0,0.0,0.0,1.0,only positive,mono,"store, web",last store purchase,"return/ ->web,store",mono purchase
775,6564,0,0.0,130.50,0.0,0.0,0.0,9.0,2023-01-03,2022-03-14,...,2.0,1.0,1.0,0.0,negative and passive,mutli,"store, web",last store purchase,"return/ ->web,store",mono purchase
776,6564,0,0.0,130.50,0.0,0.0,0.0,9.0,2023-01-03,2022-03-14,...,2.0,1.0,1.0,0.0,negative and passive,mutli,"store, web",last store purchase,"return/ ->web,store",mono purchase
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1868719,96946376,0,0.0,3245.00,0.0,0.0,0.0,40.0,2023-09-28,2023-09-15,...,1.0,0.0,0.0,1.0,only positive,mono,"store, web",last store purchase,"return/ ->web,store",mono purchase
1868889,96956644,0,0.0,31.80,0.0,0.0,0.0,4.0,2023-09-20,2023-09-20,...,1.0,0.0,0.0,1.0,only positive,mono,"store, web",last store purchase,"return/ ->web,store",mono purchase
1868905,96957695,0,0.0,1069.44,0.0,0.0,0.0,42.0,2023-09-27,2023-09-27,...,1.0,0.0,0.0,1.0,only positive,mono,"store, web",last store purchase,"return/ ->web,store",mono purchase
1868919,96958851,0,0.0,309.60,0.0,0.0,0.0,4.0,2023-10-14,2023-09-17,...,1.0,0.0,0.0,1.0,only positive,mono,"store, web",last store purchase,"return/ ->web,store",mono purchase


In [105]:
df_s_nps_lp = df_s_6.groupby(['fb_nb_type', 'purchase_type', 'last_purchase'])['user_id'].nunique().reset_index()
df_s_nps_lp.columns = ['fb_nb_type', 'purchase_type', 'last_purchase', 'nb_users']
df_s_nps_lp['share_%'] = (df_s_nps_lp['nb_users'] / df_s_nps_lp['nb_users'].sum()*100).round(2)
df_s_nps_lp

,fb_nb_type,purchase_type,last_purchase,nb_users,share_%
0,mono,only store,only store,298649,27.20
1,mono,only web,only web,204069,18.59
2,mono,"store, web",last store purchase,209022,19.04
3,mono,"store, web",last web purchase,56427,5.14
4,mutli,only store,only store,140917,12.84
5,mutli,only web,only web,37058,3.38
6,mutli,"store, web",last store purchase,121871,11.10
7,mutli,"store, web",last web purchase,29872,2.72


In [106]:
df_s_nps_gr = df_s_6.groupby(['fb_nb_type', 'purchase_type', 'last_purchase', 'return_flag'])['user_id'].nunique().reset_index()
df_s_nps_gr.columns = ['fb_nb_type', 'purchase_type', 'last_purchase', 'return_flag', 'nb_users']
df_s_nps_gr['share_%'] = (df_s_nps_gr['nb_users'] / df_s_nps_gr['nb_users'].sum()*100).round(2)
df_s_nps_gr

,fb_nb_type,purchase_type,last_purchase,return_flag,nb_users,share_%
0,mono,only store,only store,no return,66776,6.08
1,mono,only store,only store,return,231873,21.12
2,mono,only web,only web,no return,56512,5.15
3,mono,only web,only web,return,147557,13.44
4,mono,"store, web",last store purchase,no return,12076,1.10
5,mono,"store, web",last store purchase,"return/ ->web,store",138969,12.66
6,mono,"store, web",last store purchase,return/ web->store,57977,5.28
7,mono,"store, web",last web purchase,no return,8065,0.73
8,mono,"store, web",last web purchase,"return/ ->store,web",25360,2.31
9,mono,"store, web",last web purchase,return/ store->web,23002,2.10


In [149]:
df_s_nps_gr = df_s_6.groupby(['return_flag', 'purchase_nb', 'fb_nb_type', 'purchase_type', 'last_purchase', ])['user_id'].nunique().reset_index()
df_s_nps_gr.columns = ['return_flag', 'purchase_nb','fb_nb_type', 'purchase_type', 'last_purchase', 'nb_users']
df_s_nps_gr['share_%'] = (df_s_nps_gr['nb_users'] / df_s_nps_gr['nb_users'].sum()*100).round(2)
df_s_nps_gr

,return_flag,purchase_nb,fb_nb_type,purchase_type,last_purchase,nb_users,share_%
0,no return,mono purchase,mono,only web,only web,56148,5.00
1,no return,mono purchase,mono,"store, web",last store purchase,894,0.08
2,no return,mono purchase,mono,"store, web",last web purchase,961,0.09
3,no return,mono purchase,mutli,only web,only web,15941,1.42
4,no return,mono purchase,mutli,"store, web",last store purchase,747,0.07
5,no return,mono purchase,mutli,"store, web",last web purchase,634,0.06
6,no return,multi purchase,mono,only store,only store,65927,5.87
7,no return,multi purchase,mono,"store, web",last store purchase,12069,1.07
8,no return,multi purchase,mono,"store, web",last web purchase,8062,0.72
9,no return,multi purchase,mutli,only store,only store,25261,2.25


### Summary Total

In [152]:
df_s_6.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1861585 entries, 0 to 1869094
Data columns (total 26 columns):
 #   Column           Dtype  
---  ------           -----  
 0   user_id          Int64  
 1   top_l_c          Int64  
 2   amount_to_store  float64
 3   amount_to_web    float64
 4   nb_carts_store   float64
 5   nb_carts_web     float64
 6   qty_store        float64
 7   qty_web          float64
 8   min_date_store   dbdate 
 9   min_date_web     dbdate 
 10  max_date_store   dbdate 
 11  max_date_web     dbdate 
 12  min_date         dbdate 
 13  max_date         dbdate 
 14  first_nps_date   dbdate 
 15  last_nps_date    dbdate 
 16  nb_fb            float64
 17  nb_fb_neg        float64
 18  nb_fb_pas        float64
 19  nb_fb_pos        float64
 20  fb_group         object 
 21  fb_nb_type       object 
 22  purchase_type    object 
 23  last_purchase    object 
 24  return_flag      object 
 25  purchase_nb      object 
dtypes: Int64(2), dbdate(8), float64(10), objec

In [154]:
df_s_nps_res = df_s_6.groupby(['fb_nb_type', 'purchase_type','purchase_nb', 'last_purchase', 'fb_group',\
                                'return_flag'])['user_id'].nunique().reset_index()
df_s_nps_res.columns = ['fb_nb_type', 'purchase_type', 'purchase_nb', 'last_purchase','fb_group', 'return_flag', 'nb_users']
df_s_nps_res['share_%'] = (df_s_nps_res['nb_users'] / df_s_nps_res['nb_users'].sum()*100).round(2)
df_s_nps_res

,fb_nb_type,purchase_type,purchase_nb,last_purchase,fb_group,return_flag,nb_users,share_%
0,mono,only store,multi purchase,only store,only negative,no return,9950,0.89
1,mono,only store,multi purchase,only store,only negative,return,35231,3.14
2,mono,only store,multi purchase,only store,only passive,no return,16799,1.50
3,mono,only store,multi purchase,only store,only passive,return,60833,5.42
4,mono,only store,multi purchase,only store,only positive,no return,39178,3.49
...,...,...,...,...,...,...,...,...
155,mutli,"store, web",multi purchase,last web purchase,only positive,"return/ ->store,web",5763,0.51
156,mutli,"store, web",multi purchase,last web purchase,only positive,return/ store->web,5468,0.49
157,mutli,"store, web",multi purchase,last web purchase,passive and positive,no return,1357,0.12
158,mutli,"store, web",multi purchase,last web purchase,passive and positive,"return/ ->store,web",2781,0.25


In [155]:
df_s_nps_res[df_s_nps_res['return_flag']=='no return'].sort_values(by='share_%', ascending=False)

,fb_nb_type,purchase_type,purchase_nb,last_purchase,fb_group,return_flag,nb_users,share_%
4,mono,only store,multi purchase,only store,only positive,no return,39178,3.49
10,mono,only web,mono purchase,only web,only positive,no return,35023,3.12
2,mono,only store,multi purchase,only store,only passive,no return,16799,1.50
8,mono,only web,mono purchase,only web,only passive,no return,14782,1.32
58,mutli,only store,multi purchase,only store,only positive,no return,11766,1.05
0,mono,only store,multi purchase,only store,only negative,no return,9950,0.89
72,mutli,only web,mono purchase,only web,only positive,no return,8255,0.73
36,mono,"store, web",multi purchase,last store purchase,only positive,no return,7218,0.64
6,mono,only web,mono purchase,only web,only negative,no return,6343,0.56
60,mutli,only store,multi purchase,only store,passive and positive,no return,6014,0.54
